In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './Preprocessing')
import numpy as np
import os
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
from imgDataGenerator import *
from preprocessing import *
import tensorflow as tf
import keras
from keras import applications
from keras.models import Sequential, Model, Input
from keras.layers import Conv2D, MaxPooling2D, LSTM, Reshape, Permute, TimeDistributed, CuDNNLSTM,LeakyReLU
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.models import model_from_json

from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.merge import concatenate
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [ ]:
#data dimension will be: #bag * #frame * 244 * 244 * 3
#simple implementation of CNN>LSTM without multires


In [2]:
#Note for Test, Training, and Validation folders, they all need to have same categories
parentPath='/home/billy/Desktop/VideoClasBinned'
checkAndGeneratePath(parentPath)
testPath='/home/billy/Desktop/VideoClasBinned/Test'
trainPath='/home/billy/Desktop/VideoClasBinned/Training'
validPath='/home/billy/Desktop/VideoClasBinned/Validation'

trainingGen=imgDataGenerator(trainPath)
validGen=imgDataGenerator(validPath)

Length Check on Folder Succeed


In [ ]:
testPath='/home/billy/Desktop/VideoClasBinned/Test'
trainPath='/home/billy/Desktop/VideoClasBinned/Training'
validPath='/home/billy/Desktop/VideoClasBinned/Validation'
combo=prepdata(trainPath)
len(combo)
a,b=zip(*combo)
b=np.argmax(a,axis=1)
b=b.tolist()
for i in range(24):
    print(b.count(i),i)

In [3]:
#Using TimeDistributed Wrapper
inputData=Input(shape=(None, 244, 244, 3))
model = applications.inception_v3.InceptionV3(weights='imagenet', include_top=False,input_shape = ( 244, 244, 3))
#for layer in model.layers[:278]: #until mixed9_0, untrainable
#   layer.trainable = False
layer_name = 'mixed3'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
for layer in intermediate_layer_model.layers: #until mixed9_0, untrainable
  layer.trainable = False
encoded_frame = TimeDistributed(intermediate_layer_model)(inputData)

encoded_frame = TimeDistributed(Flatten())(encoded_frame)
encoded_frame = TimeDistributed(Dense(64))(encoded_frame)
encoded_frame = BatchNormalization()(encoded_frame)
encoded_frame = TimeDistributed(LeakyReLU(alpha=0.1))(encoded_frame)
intermediateOut = CuDNNLSTM(128, return_sequences = True)(encoded_frame)
intermediateOut = BatchNormalization()(intermediateOut)
# intermediateOut = CuDNNLSTM(64)(intermediateOut)
# intermediateOut = BatchNormalization()(intermediateOut)

intermediateOut = TimeDistributed(Dense(64))(intermediateOut)
intermediateOut = TimeDistributed(BatchNormalization())(intermediateOut)
intermediateOut = TimeDistributed(LeakyReLU(alpha=0.1))(intermediateOut)
intermediateOut = TimeDistributed(Dropout(0.4))(intermediateOut)

# intermediateOut = Dense(32)(intermediateOut)
# intermediateOut = BatchNormalization()(intermediateOut)
# intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)

intermediateOut = TimeDistributed(Dropout(0.4))(intermediateOut)
prediction = TimeDistributed(Dense(24, activation='softmax'))(intermediateOut)
optmr = keras.optimizers.adam(lr=0.01, decay = 5e-4)
model_final = Model(input = [inputData], output = prediction)
model_final.compile(loss = "categorical_crossentropy", optimizer = optmr, metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
model_final.summary()

In [4]:
with tf.device('GPU:0'):
    model_final.fit_generator(generator=trainingGen,
                    validation_data=validGen,
                    use_multiprocessing=True,
                    epochs = 15,
                    workers=2)
    

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
1129/1129 [==============================] - 135s 119ms/step - loss: 3.2337 - acc: 0.0637 - val_loss: 3.1279 - val_acc: 0.1470
Epoch 2/15
1129/1129 [==============================] - 128s 113ms/step - loss: 3.1723 - acc: 0.0688 - val_loss: 3.1235 - val_acc: 0.0023
Epoch 3/15
1129/1129 [==============================] - 124s 110ms/step - loss: 3.1712 - acc: 0.0629 - val_loss: 3.0847 - val_acc: 0.1818
Epoch 4/15
 385/1129 [=========>....................] - ETA: 1:23 - loss: 3.1776 - acc: 0.0563

Process ForkPoolWorker-13:
Process ForkPoolWorker-14:
Process ForkPoolWorker-16:
Process ForkPoolWorker-15:
Traceback (most recent call last):
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/billy/anaconda3/lib/

KeyboardInterrupt: 

In [ ]:
#Using TimeDistributed Wrapper and freeze all CNN layer to mix8
inputData=Input(shape=(None, 244, 244, 3))
model = applications.inception_v3.InceptionV3(weights='imagenet', include_top=False,input_shape = ( 244, 244, 3))
for layer in model.layers: #fix all param and get output from mixed8
   layer.trainable = False
layer_name = 'mixed8'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
for layer in intermediate_layer_model.layers: #fix all param and get output from mixed8
   layer.trainable = False
encoded_frame = TimeDistributed(intermediate_layer_model)(inputData)

encoded_frame=TimeDistributed(Flatten())(encoded_frame)
intermediateOut = CuDNNLSTM(128, return_sequences = True)(encoded_frame)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = CuDNNLSTM(64)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)

intermediateOut = Dense(64)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)
intermediateOut = Dropout(0.4)(intermediateOut)

intermediateOut = Dense(32)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)

intermediateOut = Dropout(0.4)(intermediateOut)
prediction = Dense(29, activation='softmax')(intermediateOut)
optmr_freeze = keras.optimizers.adam(lr=0.01, decay = 5e-4)
model_final_freeze = Model(input = [inputData], output = prediction)
model_final_freeze.compile(loss = "categorical_crossentropy", optimizer = optmr_freeze, metrics=["accuracy"])

In [ ]:
with tf.device('GPU:0'):
    model_final_freeze.fit_generator(generator=trainingGen,
                    validation_data=validGen,
                    use_multiprocessing=True,
                    epochs = 20,
                    workers=2)

In [ ]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

In [5]:
model_json = model_final.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_final.save_weights("model.h5")

In [ ]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_final = model_from_json(loaded_model_json)
# load weights into new model
model_final.load_weights("model.h5")
print("Loaded model from disk")

In [ ]:
#manually test model using testset
testGen=imgDataGenerator(testPath)
correct=0
laxcorrect=0
with tf.device('GPU:0'):
    for i in range(124):
        x,y=testGen.__getitem__(i)
        pred=model_final.predict(x)
        pred=pred.reshape((len(pred[0]),29))
        spred=np.sum(pred,axis=0)
        aspred=np.argsort(-spred)
#         print(aspred[:3])
#         print(aspred)
#         print(np.argmax(spred))
#         print(spred)
#         print(np.argmax(y[0][0]))
#         print('**')
        if(np.argmax(y[0][0]) in aspred[:2]):
            correct+=1
        if(np.argmax(y[0][0]) in aspred[:4]):
            laxcorrect+=1    
print('Accuracy of Testing Set: ',correct/124, laxcorrect/124)